In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import plotly.express as px

ModuleNotFoundError: No module named 'plotly'

In [ ]:
DATA = Path("data/processed/analytics_table.parquet")
FIGS = Path("reports/figures")
FIGS.mkdir(parents=True, exist_ok=True)

def save_fig(fig, path: Path, *, scale: int = 2) -> None:
    """Save a Plotly figure to disk (requires `kaleido`)."""
    path.parent.mkdir(parents=True, exist_ok=True)
    fig.write_image(str(path), scale=scale)

ModuleNotFoundError: No module named 'plotly'

### Questions:
1. Which countries generate the most orders and revenue, and what is the average order value (AOV) by country?
2. How does revenue change over time on a monthly basis?
3. What does the distribution of order amounts look like after winsorizing outliers?
4. Is there a statistically meaningful difference between two groups (e.g., refund rate or mean order amount) across countries?

In [ ]:
df = pd.read_parquet(DATA)

print("rows:", len(df), "cols:", len(df.columns))
print(df.dtypes.head(15))

missing = df.isna().sum().sort_values(ascending=False).head(10)
print(missing)